### En måte å dynamisk laste biblioteker

In [ ]:
# På denne måten kan man samle "alle bibliotekene man importerer hver gang" i en linje
from config.pyimports import *
# Se i filen config/pyimport hvilke biblioteker du nå har tilgang til.

In [ ]:
#pd.DataFrame()

In [ ]:
#np.nan

In [ ]:
#help(dp.execute)

Men det anbefales ikke for biblioteker du ikke trenger i *hvert ark*.

### Statiske konfigurasjonsfiler som TOML

In [ ]:
import toml

In [ ]:
config = toml.load('config.toml')

In [ ]:
paths = config['PATHS']['KAG']
paths['utpakk']

In [ ]:
# Man kan indexe i TOML-filen, som om den var en dict av dicts
config['FEATURES']['valuation']

In [ ]:
# Hvor hvert "kapittel" kan slås opp i, med videre indexing

In [ ]:
config['PATHS']['KAG']['utpakk']

In [ ]:
# I tilfelle dette ikke allerede er gjort
import dapla as dp
import pandas as pd

In [ ]:
config['PATHS']['mock']

In [ ]:
# Les inn fra dapla
pop = dp.read_pandas(
    [    p for p in # loop over
         dp.show(config['PATHS']['mock'])['path'] # pathene i mock-mappen
         if 'pop' in p # finn de som har "pop" i pathen sin
    ][0] # Ta den første av dem
)

In [ ]:
pop['sex']

In [ ]:
# Påfør formatet
pop = pop.replace(config['FORMATS']['POP'])
pop['sex']

In [ ]:
# Kod tilbake med flippete formater
reverse_omkoding = {k1 : 
                        {v2 : k2 
                         for k2, v2 
                         in v1.items() 
                         if isinstance(v2, (str, int, float))} 
                   for k1, v1 
                   in config['FORMATS']['POP'].items()}
# Vi kan ikke flippe iterables inn i keys...
reverse_omkoding
# Derfor er "AGE_GROUPS" nå blitt litt tom

In [ ]:
# Men det er jo ingen kolonne som heter "AGE_GROUPS", så det bryr vi oss ikke om her:
pop = pop.replace(reverse_omkoding)

In [ ]:
pop['sex']

### Funksjon for å påføre grupperingsvariabel utifra ett format som består av lister

In [ ]:
# Man kan ha lister i TOML, og dette åpner for "range"-formater
def range_format(df, kolonne = pd.Series(), grupperings_kolonne = '', formats = dict()):
    
    """
    Påfør range-formater som en ny kolonne, formatet bør være en dict med iterables med to verdier som values.
    """
    
    if not isinstance(df, pd.DataFrame):
        raise ValueError('First parameter must be a dataframe.')
    # Gjør masse lignende sjekker på de resterende parameterene?
    
    for k, i in formats.items():
        # For hver gruppeindeling i formatet, legg til en gruppevariabel i kolonnen "grupperings_kolonne" med dict-keyen i formatet.
        # Om det er to variabler så regner vi det som en range
        if isinstance(i, list) and len(i) == 2:
            df.loc[(kolonne >= i[0]) & (kolonne <= i[1]), grupperings_kolonne] = k
        elif isinstance(i, list):
            for j in i:
                df.loc[kolonne == j, grupperings_kolonne] = k
        # På dette tidspunktet er det potensielt feil ting sendt inn, men vi prøver litt til
        try:
            iter(i)
        except:
            # Ok da er kanskje ikke "i" iterable... Kanskje det er en enkeltverdi vi kan sette direkte?
            df.loc[kolonne == i, grupperings_kolonne] = k
            
    return df

In [ ]:
config['FORMATS']['POP']['age_ranges ']

In [ ]:
pop = range_format(pop, # Dataframe
                   pop['age'],  # Kolonne som skal tolkes
                   'age_group', # Kolonne det skal skrives til
                   config['FORMATS']['POP']['age_ranges']  # Formatet, en dict med lists
                  )

In [ ]:
# Sjekk om det funker med enkeltverdier, og lister med flere verdier.
pop[pop['age'] <= 4][['age', 'age_group']]

In [ ]:
# Om det er bare to verdier i listene, så regnes det som "ranges"
pop[['age', 'age_group']].sample(10)

### En unødvendig parser for hierarkise paths i TOMLs

In [ ]:
paths = toml.load('config/paths.toml')

In [ ]:
def buildpaths(d, depth = 0, path = '', path_list = {}):
    
    """
    Recursive function that generates a list of paths, from a nested TOML.
    Expects only the first parameter, being the loaded TOML-file. 
    Other parameters only used as defaults for the recursive nature of the function. 
    Will combine non-nested strings together, so should be only one string 'per chapter'.
    """

    # Søk etter en dict
    last_dict = {}
    for k, v in d.items():
        if isinstance(v, dict): last_dict = v
    
    # Når det ikke finnes in dict i dicten
    if not bool(last_dict):
        path += v
        #print(base)
        path_list[v] = path
        #print(depth)
        depth -= 1
    # Om det finnes en dict i dicten
    else:
        for k, v in d.items():
            # Om det vi ser på er en string
            if isinstance(v, str):
                path += v
                depth -= 1
            if isinstance(v, dict):
                depth += 1
                buildpaths(v, depth, path, path_list)
                
    return path_list

In [ ]:
paths = buildpaths(paths)

In [ ]:
paths

In [ ]:
with open('config/paths_hardcode.toml', 'w') as f:
     paths_init = toml.dump(paths, f)